# Извлечение текста из pdf

In [4]:
from pathlib import Path
import os

papers_path = Path(os.getcwd()).parent.parent / "data" / "papers"
 
papers_path

WindowsPath('c:/Users/Admin/Desktop/pythonProjects/IndexTable/data/papers')

In [7]:
import torch
print(torch.cuda.is_available())  # False значит GPU не виден
print(torch.version.cuda)         # None значит нет CUD



True
12.8


In [1]:
from tqdm import tqdm
from docling.document_converter import DocumentConverter

source = papers_path  # document per local path or URL
converter = DocumentConverter()

papers_contents = {}

for paper_path in tqdm(papers_path.iterdir()):
    result = converter.convert(paper_path)  
    paper_contents = {  
        "name": paper_path.name,
        "content": result.document.export_to_markdown()
    }
    # print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]


ModuleNotFoundError: No module named 'docling'

# Индексация в Elastic search

In [1]:
import sifts

# Для SQLite используйте префикс "sqlite:///"
collection = sifts.Collection(
    db_url="sqlite:///my_db.sqlite", 
    name="my_collection"
)

# Добавление документов
collection.add(
    contents=["Lorem ipsum dolor", "sit amet"],
    metadatas=[{"foo": "bar"}, {"foo": "baz"}],
    ids=["doc1", "doc2"],
)

# Поиск
results = collection.query("Lorem")
print(results)

{'total': 1, 'results': [{'id': 'doc1', 'content': 'Lorem ipsum dolor', 'metadata': {'foo': 'bar'}, 'rank': -9.447852760736198e-07}]}


In [36]:
import json

with open("papers_contents.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Ключи верхнего уровня:", data.keys())
print("\nПервый элемент в 'papers':")
first_key = list(data['papers'].keys())[0]
first_value = data['papers'][first_key]
print("Ключ:", first_key)
print("Значение keys():", first_value.keys())
print("Значение:", first_value)

# Теперь правильное преобразование
articles = []
for name, paper_data in data['papers'].items():
    articles.append({
        "name": name,
        "content": paper_data['content']  # контент берется из paper_data
    })

print(f"\n✅ Преобразовано {len(articles)} статей")

# Сохраняем
with open("papers_contents_new.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)

print("Сохранено в papers_contents_new.json")

Ключи верхнего уровня: dict_keys(['metadata', 'papers'])

Первый элемент в 'papers':
Ключ: Ahamed, T.; Tian, L.; Zhang, Y.; Ting, K. C. - A review of remote sensing methods for biomass feedst.pdf
Значение keys(): dict_keys(['name', 'file_size_kb', 'content_length', 'content', 'processed_at'])
Значение: {'name': 'Ahamed, T.; Tian, L.; Zhang, Y.; Ting, K. C. - A review of remote sensing methods for biomass feedst.pdf', 'file_size_kb': 1166.77, 'content_length': 84194, 'content': "<!-- image -->\n\n## Review\n\n## A review of remote sensing methods for biomass feedstock production\n\nT. Ahamed*, L. Tian, Y. Zhang, K.C. Ting\n\nEnergy Bioscience Institute, Department of Agricultural and Biological Engineering, University of Illinois at Urbana-Champaign, USA\n\n## a r t i c l e i n f o\n\nArticle history: Received 29 August 2009 Received in revised form 10 February 2011 Accepted 11 February 2011 Available online 8 April 2011\n\nKeywords: Perennial energy crops Site-specific management Veget

KeyError: 'content'

In [28]:
import pandas as pd
import json
import ast
from difflib import get_close_matches
from tqdm import tqdm

# Загружаем статьи
with open("articles.json", "r", encoding="utf8") as f:
    articles = json.load(f)

# Список всех названий в json
json_names = [a["name"] for a in articles]

# Создаём мапу название → контент
articles_map = {a["name"]: a["content"] for a in articles}

df = pd.read_excel("index_table_v1.xlsx")

for abb, name, references in tqdm(list(zip(df['abbreviation'], df['name'], df['references']))):
    try:
        ref_list = ast.literal_eval(references)
        article_names = [a['article_name'] for a in ref_list]

        for article in article_names:
            # Ищем наиболее похожее название
            match = get_close_matches(article, json_names, n=1, cutoff=0.6)

            if match:
                closest = match[0]
                content = articles_map[closest]

                print("INPUT:", article)
                print("MATCHED:", closest)
                print("CONTENT FIRST 200 CHARS:", content[:200])
                print("=" * 80)

            else:
                print("NO MATCH FOUND FOR:", article)

    except Exception as e:
        print("Error:", e)

    break



  0%|          | 0/331 [00:00<?, ?it/s]

INPUT: Gitelson, Anatoly A.; Merzlyak, Mark N.; Lichtenthaler, Hartmut K. - Detection of Red Edge Position and Chlorophyll Content by Reflectance Measurements Near 700 nm

MATCHED: Gitelson, Anatoly A.; Merzlyak, Mark N.; Lichtenthaler, Hartmut K. - Detection of Red Edge Position .pdf
CONTENT FIRST 200 CHARS: ## Introduction

The quantitative changes in chlorophyll (Chi), the main photosynthetic leaf pigment, have received relatively little attention in remote sensing studies. Several investigators have re
INPUT: le Maire, G.; Francois, C.; Dufrene, E. - Towards universal broad leaf chlorophyll indices using PROSPECT simulated database and hyperspectral reflectance measurements

MATCHED: le Maire, G.; Francois, C.; Dufrene, E. - Towards universal broad leaf chlorophyll indices using PRO.pdf
CONTENT FIRST 200 CHARS: <!-- image -->

## Abstract

Fifty-three leaves were randomly sampled on different deciduous tree species, representing a wide range of chlorophyll contents, tree ages, and 

In [3]:
В приведённых статьях найди значение минералогического индекса **{abb}**,
его полное название: **{name}**.




prompt = f"""
В приведённых статьях найди значение минералогического индекса **{abb}**,
его полное название: **{name}**.


Сгенерируй структурированное описание из трёх пунктов:
1. На что указывает этот индекс.
2. Как его интерпретировать.
3. Насколько он точный, какие есть ограничения.

СТАТЬИ:"""

SyntaxError: invalid syntax (443121447.py, line 1)

In [2]:
import pandas as pd
import json
import ast
from difflib import get_close_matches
from tqdm import tqdm
from openai import OpenAI

client = OpenAI()

# ======== LOAD DATA ========
with open("articles.json", "r", encoding="utf8") as f:
    articles = json.load(f)

json_names = [a["name"] for a in articles]
articles_map = {a["name"]: a["content"] for a in articles}

df = pd.read_excel("index_table_v1.xlsx")
df["description"] = df.get("description", None)  # если столбец уже есть — не перезаписываем


OUTPUT_FILE = "index_table_with_descriptions.xlsx"


# ======== MAIN LOOP ========
for idx, (abb, name, references) in tqdm(
        enumerate(zip(df['abbreviation'], df['name'], df['references'])),
        total=len(df)
    ):

    if idx < 227:
        continue

    try:
        ref_list = ast.literal_eval(references)
        article_titles = [a["article_name"] for a in ref_list]

        matched_contents = []

        # Ищем совпадающие статьи
        for article_title in article_titles:
            match = get_close_matches(article_title, json_names, n=1, cutoff=0.55)
            if match:
                matched_contents.append(articles_map[match[0]])

            if len(matched_contents) >= 2:
                break

        if len(matched_contents) == 0:
            df.at[idx, "description"] = "No context"
            df.to_excel(OUTPUT_FILE, index=False)  # ← Сохранение после строки
            continue

        context_articles = matched_contents[:2]

        context = "\n\n".join(context_articles)

        prompt = f"""
        В приведённых статьях найди значение минералогического индекса **{abb}**,
        его полное название: **{name}**.
    
        Пиши коротко, но полно, начинай сразу писать по пунктам, без общей информации об ответе

        Сгенерируй структурированное описание из трёх пунктов:
        1) На что указывает этот индекс?  Здесь напиши про то, какое вещество он помогает определить.
        2) Как его интерпретировать? Здесь пиши чему соовтетсвуют болшие значения индекса и есть ли соответсвие между значением индекса и концентрацией вещества?
        3) Можно ли его использоавть для поиска месторождений полезных ископаемых?. Если в статье про это не указано, просто напиши "Нет информации"

        СТАТЬИ:
        
        {context}
        """


        response = client.responses.create(
            model="gpt-5-mini",
            input=prompt,
        )

        answer = response.output_text
        df.at[idx, "description"] = answer

    except Exception as e:
        df.at[idx, "description"] = f"Error: {e}"

    # >>> Сохранение прогресса после каждой итерации <<<
    df.to_excel(OUTPUT_FILE, index=False)

    




100%|██████████| 331/331 [26:37<00:00,  4.83s/it]


In [10]:
print(answer)

Ниже — сжатое структурированное описание по индексу MD734/747/715/720 (полное название в текстах: Modified Normalized Difference 734/747/715/720), собранное из приведённых статей (в частности: Vogelmann et al., Gitelson et al., Le Maire et al.).

1) На что указывает этот индекс
- Это вегетационный индекс, чувствительный к содержанию хлорофилла (общему Chl a+b) листа/покрова — он основан на сигналах в «красном крае» (red edge).  
- Конкретно сочетает отражения в окрестностях 734–747 nm и 715–720 nm (популярная реализация — соотношение/модифицированное нормированное соотношение этих полос), что подчёркивает сдвиг и форму red‑edge, связанный с изменением содержания хлорофилла.

2) Как его интерпретировать
- Формула (как приводится в обзоре/таблицах): примерно MD734/747/715/720 ≈ (R734 / R747) / (R715 + R720) или, в близкой постановке, отношение среднего отражения в диапазоне ~734–747 nm к среднему отражению в диапазоне ~715–720 nm (варианты реализации встречаются в литературе).  
- Интерп

# Перевод названий

In [ ]:
import requests

def translate_text(texts, target_language="ru"):
    """
    Функция для перевода текста через Yandex Translate API
    
    :param texts: список строк для перевода
    :param target_language: код целевого языка (по умолчанию 'ru')
    :param folder_id: идентификатор каталога
    :param api_key: API-ключ
    :return: результат перевода
    """

    url = "https://translate.api.cloud.yandex.net/translate/v2/translate"
    
    # Тело запроса
    data = {
        "folderId": FOLDER_ID,
        "texts": texts,
        "targetLanguageCode": target_language
    }
    
    # Заголовки
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Api-Key {API_KEY}"
    }
    
    # Отправка запроса
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API Error: {response.status_code} - {response.text}")

# Пример использования
if __name__ == "__main__":
    # Ваши данные (замените на реальные)

    
    texts_to_translate = ["Hello", "World", "How are you?"]
    
    try:
        result = translate_text(
            texts=texts_to_translate,
            target_language="ru",
        )
        
        print("Перевод выполнен успешно:")
        for i, translation in enumerate(result['translations']):
            original_text = texts_to_translate[i]
            translated_text = translation['text']
            detected_lang = translation.get('detectedLanguageCode', 'unknown')
            print(f"'{original_text}' -> '{translated_text}' (язык: {detected_lang})")
            
    except Exception as e:
        print(f"Произошла ошибка: {e}")

Перевод выполнен успешно:
'Hello' -> 'Здравствуйте' (язык: en)
'World' -> 'Мир' (язык: en)
'How are you?' -> 'Как ты?' (язык: en)


In [ ]:
translate_text("Inpossible", "ru")['translations'][0]['text']

'Невозможный'

In [6]:
import pandas as pd

df = pd.read_excel("index_table_v1.xlsx")
df['name']

0       Modified Normalized Difference 734/747/715/720\r
1                                 Simple Ratio 735/710\r
2                                      Siliceous Rocks\r
3                 Chlorophyll Absorption Ratio Index 2\r
4      Normalized Difference 1094/1205 Leaf water VI 2\r
                             ...                        
326    Simple Ratio 1660/680 Disease-Water Stress Ind...
327                         Single Band 680 Blackburn1\r
328               Normalized Difference Nitrogen Index\r
329                                  Chlorophyll Green\r
330     Normalized Difference 780/550 Green NDVI hyper\r
Name: name, Length: 331, dtype: object

In [16]:
translations = []

prompt = "Ты опытный минералог, ты должен перевести минералогические индексы на русский, в ответе пиши ТОЛЬКО перевод без лишних слов, вот название индекса: "


for idx, n in enumerate(df['name']):
    translation = translate_text(n)['translations'][0]['text']
    print(translation)
    df.at[idx, "description"] = translation

Измененная нормализованная разница 734/747/715/720
Простое соотношение 735/710
Кремнистые породы
Показатель коэффициента поглощения хлорофилла 2
Нормализованная разница 1094/1205 Воды для листьев VI 2
WI/NDVI
MCARI/MTVI2
Пан НДВИ
Вмещающая порода
Нормализованная разница 531/570 Фотохимический показатель отражения 531/570
[750:850]
Однополосный 2250
Нормализованная разница 800/550 по Гринвичу NDVI hyper 2
Индекс поглощения целлюлозы 2
Взвешенный разностный растительный индекс
Однополосные 703 Буча
Индекс устойчивости растительности к атмосферным воздействиям
Фенгитический
Простое соотношение 683/510
Амфибол
Показатель отражения при потемнении
Измененное поглощение хлорофилла в показателе отражения
Однополосный 2080
Нормализованная разница 800/680 для конкретного пигмента, нормализованная разница A2, индекс Лихтенталера 1, NDVIhyper
Нормализованная разница 790/670
Простое соотношение 750/555
Нормализованный разностный индекс солености
Оптимизированный индекс растительности с поправкой на

In [18]:
df.to_excel("translated.xlsx")